### Install Geemap

In [1]:
pip install geemap

  Using cached ipyleaflet-0.19.2-py3-none-any.whl.metadata (5.3 kB)
Using cached ipyleaflet-0.19.2-py3-none-any.whl (31 kB)
  Attempting uninstall: ipyleaflet
    Found existing installation: ipyleaflet 0.17.4
    Uninstalling ipyleaflet-0.17.4:
      Successfully uninstalled ipyleaflet-0.17.4
Note: you may need to restart the kernel to use updated packages.


### Import Geemap

In [2]:
import geemap

### Verification for Earth Engine to Notebooks

In [4]:
my_map = geemap.Map()
my_map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Import Earth Engine

In [5]:
import ee

### Import Area of interest

In [6]:
NMA = ee.FeatureCollection('projects/ee-maichmarc/assets/NMA')

### Import Training Data

In [7]:
Urban = ee.FeatureCollection('projects/ee-maichmarc/assets/Urban')
Forest = ee.FeatureCollection('projects/ee-maichmarc/assets/Forest')
Agriculture = ee.FeatureCollection('projects/ee-maichmarc/assets/Agriculture')
Water = ee.FeatureCollection('projects/ee-maichmarc/assets/Water')
Rangeland = ee.FeatureCollection('projects/ee-maichmarc/assets/Rangeland')

### Import landsat 8 data for 2023

In [8]:
l8_2023_dataset = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
    .filterDate('2023-01-01', '2023-12-31')\
    .filterBounds(NMA)\
    .sort('CLOUD_COVER')\
    .first()

### Create visaualisation parameters

In [9]:
vis_params = {
  'min': 0,
  'bands': ['SR_B4', 'SR_B3', 'SR_B2']
}

### Add imported dataset to our Geemap map

In [10]:
my_map.addLayer(l8_2023_dataset, vis_params,'L8_2023')

In [11]:
my_map.centerObject(NMA)
my_map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [12]:
map_2023 = geemap.Map()

In [13]:
l8_2023_NMA = l8_2023_dataset.clip(NMA)

In [14]:
map_2023.addLayer(l8_2023_NMA, vis_params,'L8_NMA')
map_2023.centerObject(NMA)
map_2023

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Training parameters

In [15]:
label = 'Class'
bands = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7'];
inputImage = l8_2023_NMA.select(bands);

In [16]:
training = Urban.merge(Forest).merge(Agriculture).merge(Water).merge(Rangeland)

In [17]:
trainImage = inputImage.sampleRegions(
    **{'collection': training, 'properties': [label], 'scale': 30}
)

### Split training data into training and validation sets

In [18]:
trainingData = trainImage.randomColumn()
split = 0.8
trainSet = trainingData.filter(ee.Filter.lessThan('random', split))
validationSet = trainingData.filter(ee.Filter.greaterThanOrEquals('random', split))

### Train the smileCART classifier

In [19]:
classifier = ee.Classifier.smileCart().train(trainSet, label, bands)


### Classify the image

In [20]:
classified = inputImage.classify(classifier)

### Create a visualisation pallete for the classified image

In [21]:
LULC_palette = [
  'd63000', # Urban (1)
  '05450a', # Forest (2)
  'f096ff', # Agriculture (3)
  '0064c8', # Water (4)
  'ffff4c', # Rangeland (4)
  ]

### Add classified image to map

In [22]:
map_2023.centerObject(NMA)
map_2023.addLayer(classified, {'palette': LULC_palette, 'min': 1, 'max': 5},'LULC_2023')
map_2023

Map(bottom=132279.0, center=[-1.2453394220662761, 37.14196530502723], controls=(WidgetControl(options=['positi…

### Model validation

In [23]:
validated = validationSet.classify(classifier)

In [24]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '1_1_1_1_00000000000000000001_0',
 'properties': {'Class': 1,
  'SR_B1': 9194,
  'SR_B2': 10073,
  'SR_B3': 11608,
  'SR_B4': 13057,
  'SR_B5': 14942,
  'SR_B7': 17184,
  'classification': 1,
  'random': 0.9087840747219302}}

In [25]:
test_accuracy = validated.errorMatrix('Class','classification')

In [26]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0],
 [0, 20, 0, 1, 0, 7],
 [0, 0, 21, 0, 0, 0],
 [0, 1, 0, 16, 0, 0],
 [0, 0, 1, 0, 19, 0],
 [0, 1, 0, 0, 0, 26]]

In [27]:
test_accuracy.accuracy().getInfo()

0.9026548672566371

In [29]:
test_accuracy.kappa().getInfo()

0.8773072747014115

In [28]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0.7142857142857143],
 [1],
 [0.9411764705882353],
 [0.95],
 [0.9629629629629629]]

In [29]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9090909090909091,
  0.9545454545454546,
  0.9411764705882353,
  1,
  0.7878787878787878]]

### Export classified image

In [30]:
import os

In [31]:
cwd = os.getcwd() 
print(cwd)

c:\Users\MARK


In [32]:
filename_01 = os.path.join(cwd, 'NMA_LULC_2023a.tif')

In [33]:
NMA_roi = NMA.geometry()

In [34]:
geemap.ee_export_image(classified, filename=filename_01, scale=90, region=NMA_roi, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\MARK\NMA_LULC_2023a.tif
